## Fine-Tuning A Pre-Trained HuggingFace Model

Notebook Setting

- <b>Kernel</b>: pytorch 1.13 kernel
- <b>Hardware</b>: ml.c5.12xlarge Studio Notebook Instance

Dataset

- [Rotten Tomatoes Movie Review Kaggle Dataset](https://www.kaggle.com/competitions/sentiment-analysis-on-movie-reviews/data)

### Setup

In [ ]:
!pip install evaluate datasets transformers

### Dataset Preparation

In [ ]:
import pandas as pd

df = pd.read_table("reviews-dataset.tsv")
df.head()

In [ ]:
new_df = df[['Sentiment', 'Phrase']] #subset unnecessary columns
new_df

In [ ]:
new_df.columns = ["label", "text"] #adjust column names for BERT

In [ ]:
small_df = new_df.iloc[:500] #create a subset of dataframe

In [ ]:
small_df.to_csv("reviews-cleaned.csv", index=False) #create csv file with smaller dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="reviews-cleaned.csv")

In [ ]:
dataset

### Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
# Create training and evaluation datasets
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(400))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))

### Model Fine-Tuning

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()